In [1]:
import duckdb
import os
import time

In [2]:
# record start time
start = time.time()

# Read csv
duck_app = duckdb.read_csv("Google-Playstore.csv", names = ['App_Name','App_Id','Category','Rating','Rating_Count','Installs','Minimum_Installs','Maximum_Installs','Free','Price','Currency','Size','Minimum_Android','Developer_Id','Developer_Website','Developer_Email','Released','Last_Updated','Content_Rating','Privacy_Policy','Ad_Supported','In_App_Purchases','Editors_Choice','Scraped_Time'])

# record end time
end = time.time()

In [3]:
print("The time of execution of duckdb read_csv is:",(end-start) * 10**3, "ms")

The time of execution of duckdb read_csv is: 154.14857864379883 ms


In [4]:
df2 = duckdb.query("SELECT App_Name, Category, Rating, Installs, Free, Developer_Id, Released, Last_Updated FROM duck_app").to_df()

In [5]:
# Maximum number of installs
duckdb.query("SELECT distinct Installs FROM df2 order by Installs asc")

┌─────────────────┐
│    Installs     │
│     varchar     │
├─────────────────┤
│ 0+              │
│ 1+              │
│ 1,000+          │
│ 1,000,000+      │
│ 1,000,000,000+  │
│ 10+             │
│ 10,000+         │
│ 10,000,000+     │
│ 10,000,000,000+ │
│ 100+            │
│ 100,000+        │
│ 100,000,000+    │
│ 5+              │
│ 5,000+          │
│ 5,000,000+      │
│ 5,000,000,000+  │
│ 50+             │
│ 50,000+         │
│ 50,000,000+     │
│ 500+            │
│ 500,000+        │
│ 500,000,000+    │
│ NULL            │
├─────────────────┤
│     23 rows     │
└─────────────────┘

In [6]:
# Since the variable "Installs" dont show especific numbers, only agreggations, 
# we wont be able to find an exactly most downloaded app
duckdb.query("SELECT * FROM df2 WHERE Installs = '10,000,000,000+' AND Free = 'True'")

┌──────────────────────┬──────────┬────────┬─────────────────┬─────────┬──────────────┬──────────────┬──────────────┐
│       App_Name       │ Category │ Rating │    Installs     │  Free   │ Developer_Id │   Released   │ Last_Updated │
│       varchar        │ varchar  │ double │     varchar     │ boolean │   varchar    │   varchar    │   varchar    │
├──────────────────────┼──────────┼────────┼─────────────────┼─────────┼──────────────┼──────────────┼──────────────┤
│ Google Play services │ Tools    │    4.2 │ 10,000,000,000+ │ true    │ Google LLC   │ May 24, 2012 │ Jun 10, 2021 │
└──────────────────────┴──────────┴────────┴─────────────────┴─────────┴──────────────┴──────────────┴──────────────┘

In [7]:
# Since the only app with 10,000,000,000+ its the google play itself we can disconsider it 
# and use the second largest amount of downloads
duckdb.query("SELECT * FROM df2 WHERE Installs = '5,000,000,000+' AND Free = 'True' order by 'Last Updated'")

┌──────────────────────┬──────────────────────┬────────┬───┬──────────────┬──────────────┬──────────────┐
│       App_Name       │       Category       │ Rating │ … │ Developer_Id │   Released   │ Last_Updated │
│       varchar        │       varchar        │ double │   │   varchar    │   varchar    │   varchar    │
├──────────────────────┼──────────────────────┼────────┼───┼──────────────┼──────────────┼──────────────┤
│ Google TV (previou…  │ Video Players & Ed…  │    4.0 │ … │ Google LLC   │ Jun 3, 2011  │ Jun 14, 2021 │
│ WhatsApp Messenger   │ Communication        │    4.0 │ … │ WhatsApp LLC │ Oct 18, 2010 │ Jun 09, 2021 │
│ YouTube              │ Video Players & Ed…  │    4.4 │ … │ Google LLC   │ Oct 20, 2010 │ Jun 16, 2021 │
│ Google Drive         │ Productivity         │    4.3 │ … │ Google LLC   │ Apr 27, 2011 │ Jun 09, 2021 │
│ Hangouts             │ Communication        │    4.0 │ … │ Google LLC   │ May 15, 2013 │ May 14, 2021 │
│ Google Chrome: Fas…  │ Communication        

In [8]:
#New duckdb dataframe already filtred
df3 = duckdb.query("SELECT * FROM df2 WHERE Installs = '5,000,000,000+' AND Free = 'True' order by 'Last Updated'").to_df()

In [9]:
df3.shape

(14, 8)

In [10]:
#Group by "Category"
duckdb.query("SELECT Category, COUNT(Category) as TOTAL FROM df3 GROUP BY 1 ORDER BY TOTAL DESC")

┌─────────────────────────┬───────┐
│        Category         │ TOTAL │
│         varchar         │ int64 │
├─────────────────────────┼───────┤
│ Communication           │     5 │
│ Tools                   │     3 │
│ Video Players & Editors │     2 │
│ Travel & Local          │     1 │
│ Photography             │     1 │
│ Social                  │     1 │
│ Productivity            │     1 │
└─────────────────────────┴───────┘